## Library version check

In [1]:
import sys
import sktime
import tqdm as tq
import lightgbm as lgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np
import math
print("-------------------------- Python & library version --------------------------")
print("Python version: {}".format(sys.version))
print("pandas version: {}".format(pd.__version__))
print("numpy version: {}".format(np.__version__))
print("matplotlib version: {}".format(matplotlib.__version__))
print("tqdm version: {}".format(tq.__version__))
print("sktime version: {}".format(sktime.__version__))
print("lightgbm version: {}".format(lgb.__version__))
print("seaborn version: {}".format(sns.__version__))
print("scikit-learn version: {}".format(skl.__version__))
print("------------------------------------------------------------------------------")

-------------------------- Python & library version --------------------------
Python version: 3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]
pandas version: 1.5.2
numpy version: 1.21.6
matplotlib version: 3.5.2
tqdm version: 4.65.2
sktime version: 0.20.1
lightgbm version: 3.3.5
seaborn version: 0.11.2
scikit-learn version: 1.1.3
------------------------------------------------------------------------------


## 0. load the libararies

In [2]:
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from sklearn.model_selection import StratifiedKFold , KFold
from lightgbm import LGBMRegressor
from sklearn.preprocessing import LabelEncoder

pd.set_option('display.max_columns', 30)

## 1. preprocessing the data

In [3]:
train = pd.read_csv('./data/train.csv')
test = pd.read_csv('./data/test.csv')
info = pd.read_csv('./data/building_info.csv')

In [4]:
#결측치 확인(train)
def check_missing_col(dataframe):
    missing_col = []
    counted_missing_col = 0
    for i, col in enumerate(dataframe.columns):
        missing_values = sum(dataframe[col].isna())
        is_missing = True if missing_values >= 1 else False
        if is_missing:
            counted_missing_col += 1
            print(f'결측치가 있는 컬럼은: {col}입니다')
            print(f'해당 컬럼에 총 {missing_values}개의 결측치가 존재합니다.')
#            missing_col.append([col, dataframe[col].dtype])
    if counted_missing_col == 0:
        print('결측치가 존재하지 않습니다')
    return "완료"

display(check_missing_col(train))
display(check_missing_col(test))

결측치가 존재하지 않습니다


'완료'

결측치가 존재하지 않습니다


'완료'

In [5]:
#전력소비량 이상치 처리(건물번호를 기준)
for num in tqdm(range(train['건물번호'].nunique())):
    train.loc[train['건물번호'] == num+1, '전력소비량(kWh)'] = train.loc[train['건물번호'] == num+1, '전력소비량(kWh)'].clip(train.loc[train['건물번호'] == num+1, '전력소비량(kWh)'].quantile(.01), train.loc[train['건물번호'] == num+1, '전력소비량(kWh)'].quantile(.99))

100%|███████████████████████████████████████████████████████████████████████████████| 100/100 [00:00<00:00, 215.00it/s]


In [6]:
#info 메타데이터 병합
train = pd.merge(train, info[['건물번호','건물유형','연면적(m2)','냉방면적(m2)']], how = 'left', on ='건물번호')
test = pd.merge(test, info[['건물번호','건물유형','연면적(m2)','냉방면적(m2)']], how = 'left', on ='건물번호')

str_col = ['건물유형']
    
for i in str_col:
    le = LabelEncoder()
    le = le.fit(train[i])
    train[i] = le.transform(train[i])

    for label in np.unique(test[i]):
        if label not in le.classes_:
            le.classes_ = np.append(le.classes_, label)
    test[i] = le.transform(test[i])

In [7]:
## 변수들을 영문명으로 변경
cols = ['num_date_time', 'build_num', 'date_time', 'temp', 'prec', 'wind', 'hum', 'isolation', 'sunshine', 'power', 'info', 'area', 'cool_area']
train.columns = cols

def summer_cos(date):
    start_date = datetime.strptime("2024-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")
    end_date = datetime.strptime("2024-09-14 00:00:00", "%Y-%m-%d %H:%M:%S")

    period = (end_date - start_date).total_seconds()

    return math.cos(2 * math.pi * (date - start_date).total_seconds() / period)

def summer_sin(date):
    start_date = datetime.strptime("2024-06-01 00:00:00", "%Y-%m-%d %H:%M:%S")
    end_date = datetime.strptime("2024-09-14 00:00:00", "%Y-%m-%d %H:%M:%S")

    period = (end_date - start_date).total_seconds()

    return math.sin(2 * math.pi * (date - start_date).total_seconds() / period)

# 시간 관련 변수들 생성
date = pd.to_datetime(train.date_time)
train['date_time'] = pd.to_datetime(train['date_time'])
train['date'] = date.dt.date
train['hour'] = date.dt.hour
train['day'] = date.dt.day
train['weekday'] = date.dt.weekday
train['month'] = date.dt.month
train['week'] = date.dt.isocalendar().week.astype(np.int32)

# 이상치 처리
train.loc[13238:13826, 'power'] += 3500
train.loc[19161:20343, 'power'] -= 4000
train = train.drop(index=range(114240, 114408))
train = train.reset_index(drop=True)

In [8]:
import pandas as pd

def copy_pattern_by_days(
    df,
    build_num,
    target_start,
    target_end,
    offset_days,          # 예: -7(이전 주), +7(다음 주), +3(3일 뒤) 등
    col='power',
    dt_col='date_time',
    inplace=False
):
    """
    [target_start ~ target_end] 구간의 값을
    (offset_days 만큼 이동한 구간)의 패턴으로 덮어쓰기.
    """
    _df = df if inplace else df.copy()

    ts, te = pd.to_datetime(target_start), pd.to_datetime(target_end)
    ss, se = ts + pd.Timedelta(days=offset_days), te + pd.Timedelta(days=offset_days)

    m_src = (_df['build_num'] == build_num) & (_df[dt_col] >= ss) & (_df[dt_col] <= se)
    m_tgt = (_df['build_num'] == build_num) & (_df[dt_col] >= ts) & (_df[dt_col] <= te)

    src_vals = _df.loc[m_src].sort_values(dt_col)[col].values
    tgt_idx  = _df.loc[m_tgt].sort_values(dt_col).index

    if len(src_vals) == 0 or len(tgt_idx) == 0:
        return _df  # 소스/타겟이 없으면 그대로 반환

    n = min(len(src_vals), len(tgt_idx))
    _df.loc[tgt_idx[:n], col] = src_vals[:n]
    return _df


def batch_copy_patterns_by_days(
    df,
    jobs,                 # [(build_num, t_start, t_end, offset_days), ...] 또는 dict 리스트
    col='power',
    dt_col='date_time',
    inplace=False,
    verbose=False
):
    """
    여러 건을 한 번에 처리하는 배치 함수.
    jobs 원소 형태:
      - 튜플: (build_num, target_start, target_end, offset_days)
      - 딕셔너리: {
            "build_num": ...,
            "target_start": ...,
            "target_end": ...,
            # 아래 중 하나
            "offset_days": ...,
            "week_offset": ...  # 있으면 7*week_offset으로 변환
        }
      ※ offset_days가 있으면 week_offset보다 우선
    """
    def _parse(job):
        if isinstance(job, (list, tuple)) and len(job) == 4:
            b, ts, te, od = job
            return b, ts, te, od
        if isinstance(job, dict):
            b  = job['build_num']
            ts = job['target_start']
            te = job['target_end']
            if 'offset_days' in job:
                od = job['offset_days']
            elif 'week_offset' in job:
                od = 7 * job['week_offset']
            else:
                raise ValueError("dict job에는 'offset_days' 또는 'week_offset' 중 하나가 필요합니다.")
            return b, ts, te, od
        raise ValueError("jobs 항목은 (build_num, start, end, offset_days) 튜플 또는 해당 키를 가진 dict여야 합니다.")

    _df = df if inplace else df.copy()

    for job in jobs:
        b, ts, te, od = _parse(job)
        if verbose:
            print(f"[batch] build_num={b}, target=({ts}~{te}), offset_days={od}")
        _df = copy_pattern_by_days(
            _df, b, ts, te, od, col=col, dt_col=dt_col, inplace=True
        )
    return _df


jobs = [
    (5, "2024-08-04 00:00", "2024-08-04 23:00", -7),
    (6, "2024-08-15 00:00", "2024-08-15 23:00", -4),
    (6, "2024-08-16 00:00", "2024-08-16 23:00", -7),
    (6, "2024-08-17 00:00", "2024-08-17 23:00", -7),
    (6, "2024-08-18 00:00", "2024-08-18 23:00", -7),
    (7, "2024-07-07 10:00", "2024-07-08 11:00", -7),
    (8,  "2024-07-21 08:00", "2024-07-21 11:00", -7),
    (8,  "2024-08-24 00:00", "2024-08-24 23:00", -7),
    (12, "2024-07-21 00:00", "2024-07-21 23:00", +7),
    (12, "2024-08-24 00:00", "2024-08-24 23:00", -7),
    (17, "2024-06-25 15:00", "2024-06-26 09:00", -7),
    (20, "2024-06-01 00:00", "2024-06-01 23:00", +7),
    (25, "2024-07-04 12:00", "2024-07-04 14:00", +7),
    (26, "2024-06-17 14:00", "2024-06-18 11:00", -7),
    (29, "2024-06-15 22:00", "2024-06-15 23:00", -7),
    (29, "2024-06-27 00:00", "2024-06-27 01:00", -7),
    (30, "2024-08-04 00:00", "2024-08-04 23:00", -1),
    (30, "2024-08-05 00:00", "2024-08-05 23:00", -1),
    (30, "2024-08-07 00:00", "2024-08-07 23:00", -1),
    (40, "2024-07-14 00:00", "2024-07-14 01:00", -1),
    (41, "2024-06-22 01:00", "2024-06-22 04:00", -7),
    (41, "2024-07-17 00:00", "2024-07-17 23:00", -7),
    (42, "2024-07-17 00:00", "2024-07-17 23:00", -1),
    (43, "2024-06-10 17:00", "2024-06-10 18:00", -7),
    (43, "2024-08-12 16:00", "2024-08-12 17:00", -7),
    (43, "2024-07-20 00:00", "2024-07-21 23:00", -7)
]

train = batch_copy_patterns_by_days(train, jobs, col='power', dt_col='date_time', inplace=False, verbose=False)

In [9]:
def batch_interpolate_building_power(df, targets, target_col='power', method='time'):
    """
    여러 건물/시점(단일 or 구간)에 대해 시간 기반 보간을 한 번에 적용하는 함수.

    Parameters:
        df (pd.DataFrame): 전체 데이터프레임
        targets (list of tuples): 
            [(build_num, start_time), (build_num, start_time, end_time), ...] 형태의 리스트
            - end_time이 없으면 단일 시점 처리
        target_col (str): 보간할 컬럼명
        method (str): pandas.interpolate method

    Returns:
        pd.DataFrame: 보간이 적용된 원본 데이터프레임
    """
    for item in targets:
        # 튜플 길이에 따라 단일 시점/구간 처리
        if len(item) == 2:
            build_num, start_time = item
            end_time = None
        elif len(item) == 3:
            build_num, start_time, end_time = item
        else:
            raise ValueError("targets는 (build_num, start_time) 또는 (build_num, start_time, end_time) 형식이어야 합니다.")

        # 대상 건물 데이터 추출
        building = df[df['build_num'] == build_num].sort_values('date_time').copy()

        # 결측 처리
        if end_time is None:
            mask_missing = (building['date_time'] == pd.Timestamp(start_time))
        else:
            mask_missing = (
                (building['date_time'] >= pd.Timestamp(start_time)) &
                (building['date_time'] <= pd.Timestamp(end_time))
            )
        building.loc[mask_missing, target_col] = np.nan

        # 시간 기반 보간
        building.set_index('date_time', inplace=True)
        building[target_col] = building[target_col].interpolate(method=method)
        building.reset_index(inplace=True)

        # 원본 반영
        df.loc[df['build_num'] == build_num, target_col] = building[target_col].values

    return df

targets = [
    (3, '2024-07-17 14:00'),
    (7, '2024-08-06 03:00'),
    (18, '2024-07-17 14:00'),
    (30, '2024-07-13 20:00'),
    (30, '2024-07-25 00:00'),
    (42, '2024-07-17 14:00'),
    (47, '2024-07-17 14:00'),
    (55, '2024-07-17 14:00'),
    (76, '2024-08-22 21:00'),
    (81, '2024-06-27 14:00'),
    (81, '2024-07-17 14:00'),
    (82, '2024-07-17 14:00'),
    (83, '2024-07-17 14:00'), 
    (5, '2024-08-04 06:00', '2024-08-04 08:00'), 
    (18, '2024-06-11 17:00', '2024-06-11 18:00'), 
    (18, '2024-08-08 15:00', '2024-08-08 16:00'), 
    (28, '2024-07-17 14:00', '2024-07-17 15:00'), 
    (38, '2024-07-17 14:00', '2024-07-17 15:00'), 
    (41, '2024-07-17 09:00', '2024-07-17 15:00'), 
    (60, '2024-07-17 14:00', '2024-07-17 15:00'), 
    (62, '2024-07-17 13:00', '2024-07-17 15:00'), 
    (69, '2024-07-17 14:00', '2024-07-17 15:00'),  
    (76, '2024-06-20 12:00', '2024-06-20 16:00'),  
    (78, '2024-07-17 13:00', '2024-07-17 14:00'),

    # (81, '2024-07-25 13:00', '2024-07-25 17:00'), 
    # (81, '2024-07-26 13:00', '2024-07-26 17:00'), 
    # (81, '2024-07-29 13:00', '2024-07-29 17:00'), 
    # (81, '2024-07-30 13:00', '2024-07-30 17:00'), 
    # (81, '2024-08-01 13:00', '2024-08-01 17:00'), 
    # (81, '2024-08-02 13:00', '2024-08-02 17:00'), 
    # (81, '2024-08-05 13:00', '2024-08-05 17:00'), 
    # (81, '2024-08-06 13:00', '2024-08-06 16:00'), 
    # (81, '2024-08-07 13:00', '2024-08-07 17:00'), 
    # (81, '2024-08-09 13:00', '2024-08-09 17:00'), 

    # (81, '2024-08-12 10:00', '2024-08-12 15:00'), 
    # (81, '2024-08-13 13:00', '2024-08-13 17:00'), 
    # (81, '2024-08-14 10:00', '2024-08-14 17:00'), 
    # (81, '2024-08-16 10:00', '2024-08-16 17:00'), 
    # (81, '2024-08-19 10:00', '2024-08-19 17:00'), 
    # (81, '2024-08-23 11:00', '2024-08-23 16:00'), 

    (89, '2024-07-12 08:00', '2024-07-12 10:00'), 
    (97, '2024-07-17 13:00', '2024-07-17 15:00'), 
]

train = batch_interpolate_building_power(train, targets)

In [10]:
def batch_apply_pattern_scaling(df, tasks, target_col='power'):
    """
    여러 패턴 복사 작업을 한 번에 처리.
    Parameters:
        df (pd.DataFrame): 전체 데이터프레임
        tasks (list of tuples): 
            [
                (build_num, source_start, source_end, value_start, value_end, target_start, target_end),
                ...
            ]
        target_col (str): 수정할 컬럼명
    Returns:
        pd.DataFrame: 수정된 데이터프레임
    """
    for build_num, source_start, source_end, value_start_time, value_end_time, target_start, target_end in tasks:
        # 1. 원본 패턴 추출
        pattern_mask = (
            (df['build_num'] == build_num) &
            (df['date_time'] >= pd.Timestamp(source_start)) &
            (df['date_time'] <= pd.Timestamp(source_end))
        )
        P_source = df.loc[pattern_mask].sort_values('date_time')[target_col].values
        if len(P_source) == 0:
            continue  # 패턴 없으면 스킵

        # 2. 시작/종료 값
        V_start = df.loc[
            (df['build_num'] == build_num) & (df['date_time'] == pd.Timestamp(value_start_time)),
            target_col
        ].values[0]
        V_end = df.loc[
            (df['build_num'] == build_num) & (df['date_time'] == pd.Timestamp(value_end_time)),
            target_col
        ].values[0]

        # 3. 정규화 및 스케일링
        P_min, P_max = P_source.min(), P_source.max()
        P_scaled = (P_source - P_min) / (P_max - P_min + 1e-8)
        P_target = V_start + (V_end - V_start) * P_scaled

        # 4. 대상 구간 인덱스
        target_mask = (
            (df['build_num'] == build_num) &
            (df['date_time'] >= pd.Timestamp(target_start)) &
            (df['date_time'] <= pd.Timestamp(target_end))
        )
        target_indices = df.loc[target_mask].sort_values('date_time').index

        # 5. 길이 맞춰 삽입
        length = min(len(P_target), len(target_indices))
        df.loc[target_indices[:length], target_col] = P_target[:length]

    return df

tasks = [
    (7, '2024-06-30 10:00', '2024-07-01 11:00', '2024-07-07 09:00', '2024-07-08 12:00', '2024-07-07 10:00', '2024-07-08 11:00'),
    (7, '2024-07-05 14:00', '2024-07-05 23:00', '2024-07-12 13:00', '2024-07-13 00:00', '2024-07-12 14:00', '2024-07-12 23:00'),
    # (17, '2024-06-18 15:00', '2024-06-19 09:00', '2024-06-25 14:00', '2024-06-26 10:00', '2024-06-25 15:00', '2024-06-26 09:00')
]

train = batch_apply_pattern_scaling(train, tasks)

In [11]:
def batch_fill_hourly_means(df, tasks, target_col='power'):
    """
    여러 건물/기간/시간대 평균을 다른 날짜로 삽입하는 배치 함수.

    Parameters:
        df (pd.DataFrame): 전체 데이터프레임
        tasks (list of tuples): 
            [
                (build_num, source_dates, source_hours, target_date),
                ...
            ]
            - source_dates: ('start_date','end_date') or ['date1','date2',...]
            - source_hours: [hour1, hour2, ...]
            - target_date: 단일 날짜
        target_col (str): 수정할 컬럼명 (기본 'power')
    Returns:
        pd.DataFrame: 수정된 데이터프레임
    """
    for build_num, source_dates, source_hours, target_date in tasks:
        building = df[df['build_num'] == build_num].copy()

        # 날짜 마스크 생성
        if isinstance(source_dates, (tuple, list)) and len(source_dates) == 2 and not isinstance(source_dates[0], (pd.Timestamp, str)):
            # 범위일 경우
            start_date, end_date = pd.to_datetime(source_dates[0]).date(), pd.to_datetime(source_dates[1]).date()
            mask_range = (
                (building['date_time'].dt.date >= start_date) &
                (building['date_time'].dt.date <= end_date) &
                (building['date_time'].dt.hour.isin(source_hours))
            )
        else:
            # 날짜 리스트일 경우
            date_list = [pd.to_datetime(d).date() for d in source_dates]
            mask_range = (
                (building['date_time'].dt.date.isin(date_list)) &
                (building['date_time'].dt.hour.isin(source_hours))
            )

        # 시간별 평균 계산
        hourly_means = (
            building[mask_range]
            .groupby(building['date_time'].dt.hour)[target_col]
            .mean()
            .to_dict()
        )

        # 타겟 날짜에 삽입
        for hour, mean_val in hourly_means.items():
            mask_fill = (
                (df['build_num'] == build_num) &
                (df['date_time'].dt.date == pd.to_datetime(target_date).date()) &
                (df['date_time'].dt.hour == hour)
            )
            df.loc[mask_fill, target_col] = mean_val

    return df

tasks = [
    (67, ('2024-06-03', '2024-06-07'), [16, 17, 18], '2024-06-10'),          # 6/3~6/7 오후 4~6시 평균 → 6/10
    (67, ('2024-07-29', '2024-07-31'), [15, 16], '2024-08-01'),              # 7/29~7/31 오후 3~4시 평균 → 8/1
    (67, ['2024-08-13', '2024-08-14', '2024-08-16'], [16, 17], '2024-08-12'), # 8/13,14,16 오후 4~5시 평균 → 8/12
    (80, ('2024-07-01', '2024-07-05'), [11,12,13,14,19,20], '2024-07-08')    # 7/1~7/5 11~14시,19~20시 평균 → 7/8
]

train = batch_fill_hourly_means(train, tasks)

In [12]:
def scale_power_segments(train: pd.DataFrame, segments: list):
    """
    주어진 구간의 power 값을 앞뒤 하루 같은 시간대 min-max 스케일로 보정.
    보정된 값은 train['power']에 덮어씌움.

    Parameters:
        train (pd.DataFrame): 'build_num', 'date_time', 'power' 컬럼 포함 데이터프레임
        segments (list): [(build_num, start_datetime, end_datetime), ...] 형식의 튜플 리스트
    Returns:
        pd.DataFrame: 보정된 train 데이터프레임
    """
    train['date_time'] = pd.to_datetime(train['date_time'])
    
    for build_num, start_str, end_str in segments:
        target_start = pd.Timestamp(start_str)
        target_end = pd.Timestamp(end_str)

        # 대상 건물 데이터
        building_data = train[train['build_num'] == build_num].sort_values(by='date_time')
        target_mask = (building_data['date_time'] >= target_start) & (building_data['date_time'] <= target_end)
        
        # 참조 구간: 앞뒤 하루 동일 시간대
        ref_mask = (
            ((building_data['date_time'] >= target_start - pd.Timedelta(days=1)) & (building_data['date_time'] <= target_end - pd.Timedelta(days=1))) |
            ((building_data['date_time'] >= target_start + pd.Timedelta(days=1)) & (building_data['date_time'] <= target_end + pd.Timedelta(days=1)))
        )
        ref_data = building_data.loc[ref_mask, 'power']
        if ref_data.empty:
            continue  # 참조 데이터가 없으면 스킵

        ref_min, ref_max = ref_data.min(), ref_data.max()
        target_data = building_data.loc[target_mask, 'power']
        if target_data.empty or target_data.max() == target_data.min():
            continue  # 대상 데이터가 없거나 변동이 없으면 스킵

        # 스케일 조정
        scaled = (target_data - target_data.min()) / (target_data.max() - target_data.min())  # 0~1 정규화
        scaled = scaled * (ref_max - ref_min) + ref_min

        # train에 덮어쓰기
        train.loc[target_mask & (train['build_num'] == build_num), 'power'] = scaled

    return train

segments = [
    (30, '2024-06-20 06:00', '2024-06-20 23:00'),
    (30, '2024-07-06 06:00', '2024-07-06 23:00'),
]

train = scale_power_segments(train, segments)

In [13]:
def fill_with_weekly_pattern(train: pd.DataFrame, build_num: int, start_str: str, end_str: str):
    """
    특정 구간을 앞주+다음주 동일 시각 데이터 평균으로 채우고 train['power']에 덮어씀.
    """
    train['date_time'] = pd.to_datetime(train['date_time'])
    building_data = train[train['build_num'] == build_num].sort_values(by='date_time')

    # 대상 구간
    target_start = pd.Timestamp(start_str)
    target_end = pd.Timestamp(end_str)
    target_mask = (building_data['date_time'] >= target_start) & (building_data['date_time'] <= target_end)
    target_range = building_data.loc[target_mask, ['date_time']].copy()
    if target_range.empty:
        print(f"⚠️ 대상 구간({start_str}~{end_str}) 데이터 없음")
        return train

    # 앞주 & 다음주 동일 시각 데이터 가져오기
    week_offset = pd.Timedelta(days=7)
    ref1 = building_data.set_index('date_time').loc[target_start - week_offset : target_end - week_offset, ['power']].reset_index()
    ref2 = building_data.set_index('date_time').loc[target_start + week_offset : target_end + week_offset, ['power']].reset_index()

    # 두 주 패턴 align (길이가 다를 경우 보정)
    if len(ref1) != len(target_range):
        ref1 = ref1.reindex(range(len(target_range)), method='nearest')
    if len(ref2) != len(target_range):
        ref2 = ref2.reindex(range(len(target_range)), method='nearest')

    # 두 주 평균 패턴 생성
    ref_mean = (ref1['power'].values + ref2['power'].values) / 2
    target_range['power_filled'] = ref_mean

    # 덮어쓰기
    for idx, row in target_range.iterrows():
        train.loc[
            (train['build_num'] == build_num) & (train['date_time'] == row['date_time']),
            'power'
        ] = row['power_filled']

    return train

# 7월 20일 02시 ~ 7월 22일 10시, 건물 49
train = fill_with_weekly_pattern(
    train, 
    build_num=43, 
    start_str="2024-07-20 02:00", 
    end_str="2024-07-22 10:00"
)

train = fill_with_weekly_pattern(
    train, 
    build_num=53, 
    start_str="2024-06-14 16:00", 
    end_str="2024-06-17 09:00"
)

train = fill_with_weekly_pattern(
    train, 
    build_num=67, 
    start_str="2024-07-27 00:00", 
    end_str="2024-07-28 00:00"
)

train = fill_with_weekly_pattern(
    train, 
    build_num=94, 
    start_str="2024-07-27 00:00", 
    end_str="2024-07-28 00:00"
)

In [14]:
def add_value_to_segment(train: pd.DataFrame, build_num: int, start_str: str, end_str: str, add_value: float):
    """
    특정 건물의 지정 구간에 일정 값을 더해 train['power']에 덮어씀.
    """
    train['date_time'] = pd.to_datetime(train['date_time'])
    target_start = pd.Timestamp(start_str)
    target_end = pd.Timestamp(end_str)

    mask = (
        (train['build_num'] == build_num) &
        (train['date_time'] >= target_start) &
        (train['date_time'] <= target_end)
    )

    train.loc[mask, 'power'] = train.loc[mask, 'power'] + add_value
    return train

train = add_value_to_segment(
    train,
    build_num=53,
    start_str="2024-08-18 16:00",
    end_str="2024-08-19 07:00",
    add_value=400
)

train = add_value_to_segment(
    train,
    build_num=67,
    start_str="2024-06-01 00:00",
    end_str="2024-06-03 09:00",
    add_value=780
)

# train = add_value_to_segment(
#     train,
#     build_num=10,
#     start_str="2024-06-01 00:00",
#     end_str="2024-07-04 07:00",
#     add_value=900
# )

In [15]:
def fill_with_prev_next_day_avg(train: pd.DataFrame, build_num: int, start_str: str, end_str: str):
    """
    특정 건물의 지정 구간을 하루 전/하루 뒤 동일 시간대의 평균 값으로 채움.
    train['power']에 덮어씀.
    """
    train['date_time'] = pd.to_datetime(train['date_time'])
    building_data = train[train['build_num'] == build_num].sort_values(by='date_time')

    # 대상 구간
    target_start = pd.Timestamp(start_str)
    target_end = pd.Timestamp(end_str)
    target_mask = (building_data['date_time'] >= target_start) & (building_data['date_time'] <= target_end)
    target_times = building_data.loc[target_mask, 'date_time']
    if target_times.empty:
        print(f"⚠️ 대상 구간({start_str}~{end_str}) 데이터 없음")
        return train

    # 하루 전/하루 뒤 동일 시간대 구간
    prev_day_mask = (building_data['date_time'] >= target_start - pd.Timedelta(days=1)) & (building_data['date_time'] <= target_end - pd.Timedelta(days=1))
    next_day_mask = (building_data['date_time'] >= target_start + pd.Timedelta(days=1)) & (building_data['date_time'] <= target_end + pd.Timedelta(days=1))
    prev_data = building_data.loc[prev_day_mask, ['date_time', 'power']]
    next_data = building_data.loc[next_day_mask, ['date_time', 'power']]

    if prev_data.empty and next_data.empty:
        print(f"⚠️ 참조 데이터 없음({start_str}~{end_str})")
        return train

    # 평균 패턴 계산 (있으면 합쳐서 평균)
    ref_values = []
    if not prev_data.empty:
        ref_values.append(prev_data['power'].values)
    if not next_data.empty:
        ref_values.append(next_data['power'].values)
    ref_mean = sum(ref_values) / len(ref_values)  # 두 날짜 평균

    # 길이가 다르면 맞춰서 채움
    ref_mean_series = pd.Series(ref_mean)
    ref_mean_series = ref_mean_series.reindex(range(len(target_times)), method='nearest')

    # 덮어쓰기
    train.loc[
        (train['build_num'] == build_num) & (train['date_time'] >= target_start) & (train['date_time'] <= target_end),
        'power'
    ] = ref_mean_series.values

    return train

train = fill_with_prev_next_day_avg(
    train,
    build_num=70,
    start_str="2024-06-04 09:00",
    end_str="2024-06-05 09:00"
)

In [16]:
def fill_power_with_holiday_pattern(train, build_num, date_ranges):
    """
    특정 건물의 지정 날짜 구간 전력 사용량을
    전후 holiday 패턴(같은 시간대 평균)으로 대체하는 함수.

    Parameters:
        train (pd.DataFrame): 전체 데이터프레임
        build_num (int): 건물 번호
        date_ranges (list of tuples): [(start_date, end_date), ...] 형식의 구간 리스트 (문자열 or Timestamp)
    Returns:
        pd.DataFrame: power가 덮어씌워진 원본 train DataFrame
    """
    # 데이터 정렬 및 시간 변환
    train['date_time'] = pd.to_datetime(train['date_time'])
    building_data = train[train['build_num'] == build_num].sort_values(by='date_time').copy()
    building_data['hour'] = building_data['date_time'].dt.hour

    # holiday 날짜
    holiday_dates = building_data[building_data['holiday'] == 1]['date_time']

    for start_date, end_date in date_ranges:
        target_start = pd.Timestamp(start_date)
        target_end = pd.Timestamp(end_date)

        # 전후 holiday 추출
        prev_holiday = holiday_dates[holiday_dates < target_start].max()
        next_holiday = holiday_dates[holiday_dates > target_end].min()
        if pd.isna(prev_holiday) or pd.isna(next_holiday):
            continue  # holiday 없으면 skip

        # 전후 holiday 패턴
        prev_pattern = building_data[building_data['date_time'].dt.date == prev_holiday.date()]
        next_pattern = building_data[building_data['date_time'].dt.date == next_holiday.date()]
        holiday_pattern = (prev_pattern.groupby('hour')['power'].mean() +
                           next_pattern.groupby('hour')['power'].mean()) / 2

        # 대체
        target_mask = (building_data['date_time'] >= target_start) & (building_data['date_time'] <= target_end)
        building_data.loc[target_mask, 'power'] = building_data.loc[target_mask].apply(
            lambda row: holiday_pattern.loc[row['hour']] if row['hour'] in holiday_pattern.index else row['power'],
            axis=1
        )

    # train에 반영
    train.loc[building_data.index, 'power'] = building_data['power']
    return train

In [17]:
isolation = pd.pivot_table(train, values = 'isolation', index = ['build_num', 'hour', 'month'], aggfunc = np.mean).reset_index()
sunshine = pd.pivot_table(train, values = 'sunshine', index = ['build_num', 'hour', 'month'], aggfunc = np.mean).reset_index()

# 건물별, 요일별, 시간별 전력소비량 평균
power_mean_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'weekday'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['target_mean_1'] = train.progress_apply(lambda x : power_mean_1.loc[(power_mean_1.build_num == x['build_num']) & (power_mean_1.hour == x['hour']) & (power_mean_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

# 건물별, 요일별, 시간별 전력소비량 표준편차
power_std_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'weekday'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['target_std_1'] = train.progress_apply(lambda x : power_std_1.loc[(power_std_1.build_num == x['build_num']) & (power_std_1.hour == x['hour']) & (power_std_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

## 건물별, 요일별, 시간별 전력소비량 최소
power_min_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'weekday'], aggfunc = np.min).reset_index()
tqdm.pandas()
train['target_min_1'] = train.progress_apply(lambda x : power_min_1.loc[(power_min_1.build_num == x['build_num']) & (power_min_1.hour == x['hour']) & (power_min_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

## 건물별, 요일별, 시간별 전력소비량 최대
power_max_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'weekday'], aggfunc = np.max).reset_index()
tqdm.pandas()
train['target_max_1'] = train.progress_apply(lambda x : power_max_1.loc[(power_max_1.build_num == x['build_num']) & (power_max_1.hour == x['hour']) & (power_max_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

# 건물별, 시간별 전력소비량 평균
hour_mean_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['hour_mean_1'] = train.progress_apply(lambda x : hour_mean_1.loc[(hour_mean_1.build_num == x['build_num']) & (hour_mean_1.hour == x['hour']) ,'power'].values[0], axis = 1)

# 건물별, 시간별 전력소비량 표준편차
hour_std_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['hour_std_1'] = train.progress_apply(lambda x : hour_std_1.loc[(hour_std_1.build_num == x['build_num']) & (hour_std_1.hour == x['hour']) ,'power'].values[0], axis = 1)

## 공휴일 변수 추가
train['holiday'] = train.apply(lambda x : 0 if x['weekday']<5 else 1, axis = 1)
train.loc[('20240606'<=train.date_time)&(train.date_time<'20240607'),'holiday'] = 1
train.loc[('20240815'<=train.date_time)&(train.date_time<'20240816'),'holiday'] = 1

# 규칙 정의 함수
def apply_holiday_rules(row):
    bn = row['build_num']
    wd = row['weekday']
    day = row['day']
    week = (row['day'] - 1) // 7 + 1  # 몇째 주인지 계산

    # 📌 개별 규칙 적용
    if bn == 2:   # 상용: 토요일 쉼 → holiday = 1 if 토요일 else 0
        return 1 if wd == 5 else 0
    elif bn == 7:   # 건물기타: 일요일 쉼
        return 1 if wd == 6 else 0
    elif bn == 18:  # 백화점: 일요일 쉼
        return 1 if wd == 6 else 0
#     elif bn == 19:  # 백화점: 둘째주 월요일 쉼
#         return 1 if wd == 0 and week == 2 else 0
    elif bn == 25:  # 아파트: 토요일에 적게 씀 (평일로 간주)
        return 0
    elif bn == 26:  # 건물기타: 주말에 더 씀 → 평일로 간주
        return 0 if wd in [5, 6] else 1
    elif bn == 27:  # 백화점: 둘째, 넷째주 일요일 쉼
        return 1 if wd == 6 and week in [2, 4] else 0
    elif bn == 29:  # 백화점: 매달 10일 쉼
        return 1 if day == 10 else 0
    elif bn == 31:  # 아파트: 휴일 없음 → 평일 취급
        return 0
    elif bn == 32:  # 백화점: 둘째, 넷째주 월요일 쉼
        return 1 if wd == 0 and week in [2, 4] else 0
    elif bn == 34:  # 백화점: 휴일 없음 → 평일 취급
        return 0
    elif bn == 35:  # 전화국: 휴일 없음 -> 평일 취급
        return 0
    elif bn == 36:  # 전화국: 휴일 없음 -> 평일 취급
        return 1 if wd in [5, 6] else 0
    elif bn == 40:  # 백화점: 둘째, 넷째주 월요일 쉼
        return 1 if wd == 6 and week in [2, 4] else 0
    elif bn == 41:
        return 0
#     elif bn == 45:
#         return 1 if day == 10 else 0
    elif bn == 54:
        return 0
    elif bn == 57:
        return 0
    elif bn == 58:
        return 0
    elif bn == 59:
        return 1 if wd == 6 and week in [2, 4] else 0
    elif bn == 61:
        return 0
    elif bn == 63:
        return 1 if wd == 6 and week in [2, 4] else 0
    elif bn in [97]:  # 토요일쉼
        return 1 if wd == 5 else 0
    elif bn in [1,4,9,10,11,19,28,30,33,45,65,70,71,73,74,76,77,78,79,82,84,85,88,89,91,92,93,95,96,98,99,100]:
        return 0
    else:
        # 기본 규칙 유지
        return row['holiday']

# 규칙 적용
train['holiday'] = train.apply(apply_holiday_rules, axis=1)

single_day_holidays = [
    (19, '2024-06-10'),
    (19, '2024-07-08'),
    (19, '2024-08-19'),
    (23, '2024-06-07'),
    (23, '2024-08-16'),
    (29, '2024-06-23'),
    (29, '2024-07-28'),
    (45, '2024-06-10'),
    (45, '2024-07-08'),
    (45, '2024-08-19'),
    (49, '2024-08-22'),
    (54, '2024-06-17'),
    (54, '2024-07-01'),
    (54, '2024-08-19'),
    (67, '2024-07-26'),
    (79, '2024-06-17'),
    (79, '2024-07-01'),
    (79, '2024-08-19'),
    (94, '2024-06-07'),
    (94, '2024-08-16'),
    (95, '2024-07-08'),
    (95, '2024-08-05'),
]

for build_num, date_str in single_day_holidays:
    target_date = pd.to_datetime(date_str).date()  # 날짜만 비교
    train.loc[
        (train['build_num'] == build_num) & 
        (train['date_time'].dt.date == target_date),
        'holiday'
    ] = 1

single_day_no_holiday = [
    (67, '2024-06-06')
]

for build_num, date_str in single_day_no_holiday:
    target_date = pd.to_datetime(date_str).date()  # 날짜만 비교
    train.loc[
        (train['build_num'] == build_num) & 
        (train['date_time'].dt.date == target_date),
        'holiday'
    ] = 0

date_ranges = [
    ('2024-07-26 00:00:00', '2024-07-26 23:59:59'),
]
train = fill_power_with_holiday_pattern(train, build_num=67, date_ranges=date_ranges)
    
# 건물별, 휴일별, 시간별 전력소비량 평균
power_holiday_mean_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'holiday'], aggfunc = np.mean).reset_index()
tqdm.pandas()
train['holiday_mean_1'] = train.progress_apply(lambda x : power_holiday_mean_1.loc[(power_holiday_mean_1.build_num == x['build_num']) & (power_holiday_mean_1.hour == x['hour']) & (power_holiday_mean_1.holiday == x['holiday']) ,'power'].values[0], axis = 1)

# 건물별, 휴일별, 시간별 전력소비량 표준편차
power_holiday_std_1 = pd.pivot_table(train, values = 'power', index = ['build_num', 'hour', 'holiday'], aggfunc = np.std).reset_index()
tqdm.pandas()
train['holiday_std_1'] = train.progress_apply(lambda x : power_holiday_std_1.loc[(power_holiday_std_1.build_num == x['build_num']) & (power_holiday_std_1.hour == x['hour']) & (power_holiday_std_1.holiday == x['holiday']) ,'power'].values[0], axis = 1)
    
## https://dacon.io/competitions/official/235680/codeshare/2366?page=1&dtype=recent
train['sin_hour'] = np.sin(2*np.pi*train.hour/24)
train['cos_hour'] = np.cos(2*np.pi*train.hour/24)
train['sin_date'] = -np.sin(2 * np.pi * (train['month']+train['day']/31)/12)
train['cos_date'] = -np.cos(2 * np.pi * (train['month']+train['day']/31)/12)
train['sin_month'] = -np.sin(2 * np.pi * train['month']/12)
train['cos_month'] = -np.cos(2 * np.pi * train['month']/12)
train['sin_weekday'] = -np.sin(2 * np.pi * (train['weekday']+1)/7)
train['cos_weekday'] = -np.cos(2 * np.pi * (train['weekday']+1)/7)

#summer_sin, cos
train['summer_sin'] = train['date_time'].apply(summer_sin)
train['summer_cos'] = train['date_time'].apply(summer_cos)

## 화씨 온도
train['temp_F'] = (train['temp'] * 9/5) + 32 

## 체감 온도
train['WC']=13.12+0.6215*train['temp']-13.947*train['wind']**0.16+0.486*train['temp']*train['wind']**0.16

## 불쾌 지수
train['THI'] = 9/5*train['temp'] - 0.55*(1-train['hum']/100)*(9/5*train['hum']-26)+32

train['dew_point'] = train['temp'] - (100 - train['hum']) / 5

def CDH(xs):
    ys = []
    for i in range(len(xs)):
        if i < 11:
            ys.append(np.sum(xs[:(i+1)]-26))
        else:
            ys.append(np.sum(xs[(i-11):(i+1)]-26))
    return np.array(ys)

cdhs = np.array([])
for num in range(1,101,1):
    temp = train[train['build_num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
train['CDH'] = cdhs

## mean temperature
train = train.merge(train.groupby(['build_num','date'])['temp'].mean().reset_index().rename(columns = {'temp':'mean_temp'}), on = ['build_num','date'], how = 'left')

## min temperature
train = train.merge(train.groupby(['build_num','date'])['temp'].min().reset_index().rename(columns = {'temp':'min_temp'}), on = ['build_num','date'], how = 'left')

## max temperature
train = train.merge(train.groupby(['build_num','date'])['temp'].max().reset_index().rename(columns = {'temp':'max_temp'}), on = ['build_num','date'], how = 'left')

## mean windspeed
train = train.merge(train.groupby(['build_num','date'])['wind'].mean().reset_index().rename(columns = {'wind':'mean_wind'}), on = ['build_num','date'], how = 'left')

# ## min windspeed
# train = train.merge(train.groupby(['build_num','date'])['wind'].min().reset_index().rename(columns = {'wind':'min_wind'}), on = ['build_num','date'], how = 'left')

# ## max windspeed
# train = train.merge(train.groupby(['build_num','date'])['wind'].max().reset_index().rename(columns = {'wind':'max_wind'}), on = ['build_num','date'], how = 'left')

## mean humidity
train = train.merge(train.groupby(['build_num','date'])['hum'].mean().reset_index().rename(columns = {'hum':'mean_hum'}), on = ['build_num','date'], how = 'left')

# ## min humidity
# train = train.merge(train.groupby(['build_num','date'])['hum'].min().reset_index().rename(columns = {'hum':'min_hum'}), on = ['build_num','date'], how = 'left')

# ## max humidity
# train = train.merge(train.groupby(['build_num','date'])['hum'].max().reset_index().rename(columns = {'hum':'max_hum'}), on = ['build_num','date'], how = 'left')

## mean THI
train = train.merge(train.groupby(['build_num','date'])['THI'].mean().reset_index().rename(columns = {'THI':'mean_THI'}), on = ['build_num','date'], how = 'left')

## mean CDH
train = train.merge(train.groupby(['build_num','date'])['CDH'].mean().reset_index().rename(columns = {'CDH':'mean_CDH'}), on = ['build_num','date'], how = 'left')

## mean WC
train = train.merge(train.groupby(['build_num','date'])['WC'].mean().reset_index().rename(columns = {'WC':'mean_WC'}), on = ['build_num','date'], how = 'left')

## z-score
train['z_score'] = train['target_mean_1'] / train['target_std_1']

## temp_diff
train['temp_diff'] = train['max_temp'] - train['min_temp']

train.drop(columns=['date','day'], inplace=True)
train.head()

100%|████████████████████████████████████████████████████████████████████████| 203832/203832 [01:48<00:00, 1880.91it/s]


,num_date_time,build_num,date_time,temp,prec,wind,hum,isolation,sunshine,power,info,area,cool_area,hour,weekday,...,temp_F,WC,THI,dew_point,CDH,mean_temp,min_temp,max_temp,mean_wind,mean_hum,mean_THI,mean_CDH,mean_WC,z_score,temp_diff
0,1_20240601 00,1,2024-06-01 00:00:00,18.3,0.0,2.6,82.0,0.0,0.0,5794.80,9,82912.71,77586.0,0,5,...,64.94,18.605526,52.9016,14.7,-7.7,20.570833,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,11.353622,7.2
1,1_20240601 01,1,2024-06-01 01:00:00,18.3,0.0,2.7,82.0,0.0,0.0,5591.85,9,82912.71,77586.0,1,5,...,64.94,18.569864,52.9016,14.7,-15.4,20.570833,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,10.540611,7.2
2,1_20240601 02,1,2024-06-01 02:00:00,18.1,0.0,2.6,80.0,0.0,0.0,5338.17,9,82912.71,77586.0,2,5,...,64.58,18.367969,51.6000,14.1,-23.3,20.570833,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,9.252619,7.2
3,1_20240601 03,1,2024-06-01 03:00:00,18.0,0.0,2.6,81.0,0.0,0.0,4554.42,9,82912.71,77586.0,3,5,...,64.40,18.249191,51.8809,14.2,-31.3,20.570833,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,7.118597,7.2
4,1_20240601 04,1,2024-06-01 04:00:00,17.8,0.0,1.3,81.0,0.0,0.0,3602.25,9,82912.71,77586.0,4,5,...,64.04,18.659442,51.5209,14.0,-39.5,20.570833,17.6,24.8,2.75,63.375,53.788788,-53.0625,21.342341,7.868436,7.2


In [18]:
# train set과 동일한 전처리 과정
test['일시'] = pd.to_datetime(test['일시'])
cols = ['num_date_time', 'build_num', 'date_time', 'temp' , 'prec', 'wind', 'hum', 'info', 'area', 'cool_area']
test.columns = cols

# 시간 관련 변수들 생성
date = pd.to_datetime(test.date_time)
test['date'] = date.dt.date
test['hour'] = date.dt.hour
test['day'] = date.dt.day
test['weekday'] = date.dt.weekday
test['month'] = date.dt.month
test['week'] = date.dt.isocalendar().week.astype(np.int32)

test['holiday'] = test.apply(lambda x : 0 if x['weekday']<5 else 1, axis = 1)
test['holiday'] = test.apply(apply_holiday_rules, axis=1)

## 일조
tqdm.pandas()
test['isolation'] = np.round(test.progress_apply(lambda x : isolation.loc[(isolation.build_num == x['build_num']) & (isolation.hour == x['hour']) & (isolation.month == x['month']) ,'isolation'].values[0], axis = 1), 1)

## 일사
tqdm.pandas()
test['sunshine'] = np.round(test.progress_apply(lambda x : sunshine.loc[(sunshine.build_num == x['build_num']) & (sunshine.hour == x['hour']) & (sunshine.month == x['month']) ,'sunshine'].values[0], axis = 1), 2)

# 건물별, 요일별, 시간별 전력소비량 평균
tqdm.pandas()
test['target_mean_1'] = test.progress_apply(lambda x : power_mean_1.loc[(power_mean_1.build_num == x['build_num']) & (power_mean_1.hour == x['hour']) & (power_mean_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

# 건물별, 요일별, 시간별 전력소비량 표준편차
tqdm.pandas()
test['target_std_1'] = test.progress_apply(lambda x : power_std_1.loc[(power_std_1.build_num == x['build_num']) & (power_std_1.hour == x['hour']) & (power_std_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['target_min_1'] = test.progress_apply(lambda x : power_min_1.loc[(power_min_1.build_num == x['build_num']) & (power_min_1.hour == x['hour']) & (power_min_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['target_max_1'] = test.progress_apply(lambda x : power_max_1.loc[(power_max_1.build_num == x['build_num']) & (power_max_1.hour == x['hour']) & (power_max_1.weekday == x['weekday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['holiday_mean_1'] = test.progress_apply(lambda x : power_holiday_mean_1.loc[(power_holiday_mean_1.build_num == x['build_num']) & (power_holiday_mean_1.hour == x['hour']) & (power_holiday_mean_1.holiday == x['holiday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['holiday_std_1'] = test.progress_apply(lambda x : power_holiday_std_1.loc[(power_holiday_std_1.build_num == x['build_num']) & (power_holiday_std_1.hour == x['hour']) & (power_holiday_std_1.holiday == x['holiday']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['hour_mean_1'] = test.progress_apply(lambda x : hour_mean_1.loc[(hour_mean_1.build_num == x['build_num']) & (hour_mean_1.hour == x['hour']) ,'power'].values[0], axis = 1)

tqdm.pandas()
test['hour_std_1'] = test.progress_apply(lambda x : hour_std_1.loc[(hour_std_1.build_num == x['build_num']) & (hour_std_1.hour == x['hour']) ,'power'].values[0], axis = 1)

test['sin_hour'] = np.sin(2*np.pi*test.hour/24)
test['cos_hour'] = np.cos(2*np.pi*test.hour/24)
test['sin_date'] = -np.sin(2 * np.pi * (test['month']+test['day']/31)/12)
test['cos_date'] = -np.cos(2 * np.pi * (test['month']+test['day']/31)/12)
test['sin_month'] = -np.sin(2 * np.pi * test['month']/12)
test['cos_month'] = -np.cos(2 * np.pi * test['month']/12)
test['sin_weekday'] = -np.sin(2 * np.pi * (test['weekday']+1)/7.0)
test['cos_weekday'] = -np.cos(2 * np.pi * (test['weekday']+1)/7.0)

#summer_sin, cos
test['summer_sin'] = test['date_time'].apply(summer_sin)
test['summer_cos'] = test['date_time'].apply(summer_cos)

## 화씨 온도
test['temp_F'] = (test['temp'] * 9/5) + 32 

## 체감 온도
test['WC']=13.12+0.6215*test['temp']-13.947*test['wind']**0.16+0.486*test['temp']*test['wind']**0.16

test['THI'] = 9/5*test['temp'] - 0.55*(1-test['hum']/100)*(9/5*test['hum']-26)+32

test['dew_point'] = test['temp'] - (100 - test['hum']) / 5

cdhs = np.array([])
for num in range(1,101,1):
    temp = test[test['build_num'] == num]
    cdh = CDH(temp['temp'].values)
    cdhs = np.concatenate([cdhs, cdh])
test['CDH'] = cdhs

## mean temperature
test = test.merge(test.groupby(['build_num','date'])['temp'].mean().reset_index().rename(columns = {'temp':'mean_temp'}), on = ['build_num','date'], how = 'left')

## min temperature
test = test.merge(test.groupby(['build_num','date'])['temp'].min().reset_index().rename(columns = {'temp':'min_temp'}), on = ['build_num','date'], how = 'left')

## max temperature
test = test.merge(test.groupby(['build_num','date'])['temp'].max().reset_index().rename(columns = {'temp':'max_temp'}), on = ['build_num','date'], how = 'left')

## mean windspeed
test = test.merge(test.groupby(['build_num','date'])['wind'].mean().reset_index().rename(columns = {'wind':'mean_wind'}), on = ['build_num','date'], how = 'left')

# ## min windspeed
# test = test.merge(test.groupby(['build_num','date'])['wind'].min().reset_index().rename(columns = {'wind':'min_wind'}), on = ['build_num','date'], how = 'left')

# ## max windspeed
# test = test.merge(test.groupby(['build_num','date'])['wind'].max().reset_index().rename(columns = {'wind':'max_wind'}), on = ['build_num','date'], how = 'left')

## mean humidity
test = test.merge(test.groupby(['build_num','date'])['hum'].mean().reset_index().rename(columns = {'hum':'mean_hum'}), on = ['build_num','date'], how = 'left')

# ## min humidity
# test = test.merge(test.groupby(['build_num','date'])['hum'].min().reset_index().rename(columns = {'hum':'min_hum'}), on = ['build_num','date'], how = 'left')

# ## max humidity
# test = test.merge(test.groupby(['build_num','date'])['hum'].max().reset_index().rename(columns = {'hum':'max_hum'}), on = ['build_num','date'], how = 'left')

## mean THI
test = test.merge(test.groupby(['build_num','date'])['THI'].mean().reset_index().rename(columns = {'THI':'mean_THI'}), on = ['build_num','date'], how = 'left')

## mean CDH
test = test.merge(test.groupby(['build_num','date'])['CDH'].mean().reset_index().rename(columns = {'CDH':'mean_CDH'}), on = ['build_num','date'], how = 'left')

## mean WC
test = test.merge(test.groupby(['build_num','date'])['WC'].mean().reset_index().rename(columns = {'WC':'mean_WC'}), on = ['build_num','date'], how = 'left')

## z-score
test['z_score'] = test['target_mean_1'] / test['target_std_1']

## temp_diff
test['temp_diff'] = test['max_temp'] - test['min_temp']

test.drop(['date_time','date','day'], axis = 1, inplace = True)
test.head()

100%|██████████████████████████████████████████████████████████████████████████| 16800/16800 [00:06<00:00, 2644.18it/s]


,num_date_time,build_num,temp,prec,wind,hum,info,area,cool_area,hour,weekday,month,week,holiday,isolation,...,temp_F,WC,THI,dew_point,CDH,mean_temp,min_temp,max_temp,mean_wind,mean_hum,mean_THI,mean_CDH,mean_WC,z_score,temp_diff
0,1_20240825 00,1,26.5,0.0,0.7,80.0,9,82912.71,77586.0,0,6,8,34,0,0.0,...,79.70,28.580992,66.7200,22.5,0.5,28.408333,25.0,32.6,1.458333,76.25,69.47355,18.695833,30.722386,7.365460,7.6
1,1_20240825 01,1,26.1,0.0,0.0,80.0,9,82912.71,77586.0,1,6,8,34,0,0.0,...,78.98,29.341150,66.0000,22.1,0.6,28.408333,25.0,32.6,1.458333,76.25,69.47355,18.695833,30.722386,5.735023,7.6
2,1_20240825 02,1,25.9,0.0,0.3,83.0,9,82912.71,77586.0,2,6,8,34,0,0.0,...,78.62,28.095476,67.0821,22.5,0.5,28.408333,25.0,32.6,1.458333,76.25,69.47355,18.695833,30.722386,6.212821,7.6
3,1_20240825 03,1,25.7,0.0,1.1,83.0,9,82912.71,77586.0,3,6,8,34,0,0.0,...,78.26,27.613364,66.7221,22.3,0.2,28.408333,25.0,32.6,1.458333,76.25,69.47355,18.695833,30.722386,5.765610,7.6
4,1_20240825 04,1,25.5,0.0,1.0,86.0,9,82912.71,77586.0,4,6,8,34,0,0.0,...,77.90,27.414250,67.9824,22.7,-0.3,28.408333,25.0,32.6,1.458333,76.25,69.47355,18.695833,30.722386,8.877454,7.6


In [19]:
test['hum'] = test['hum'].astype(float)

In [20]:
x_train = train.drop(columns=['num_date_time','date_time','power'])
y_train = train['power'].values
x_test = test.drop(columns=['num_date_time'])

In [21]:
x_test = x_test[x_train.columns]

## 2. Model : LGBM

In [22]:
def SMAPE(true, pred):
    return np.mean((np.abs(true-pred))/(np.abs(true) + np.abs(pred))) * 200

def smape(preds, target):
    '''
    Function to calculate SMAPE
    '''
    n = len(preds)
    masked_arr = ~((preds==0)&(target==0))
    preds, target = preds[masked_arr], target[masked_arr]
    num = np.abs(preds-target)
    denom = np.abs(preds)+np.abs(target)
    smape_val = (200*np.sum(num/denom))/n
    return smape_val

def lgbm_smape(preds, train_data):
    '''
    Custom Evaluation Function for LGBM
    '''
    # labels = train_data.get_label()
    labels = train_data
    smape_val = smape(preds, labels)
    return 'SMAPE', smape_val, False

In [23]:
cat_features = ['build_num','hour','month','weekday','holiday']

is_holdout = False
iterations = 80000
patience = 300

In [24]:
models_l2 = []

n_split_list = [10]
smape_l2_score = []

for split in n_split_list:
    cv = StratifiedKFold(n_splits=split, shuffle=True, random_state=42)
    fold = 1
    for tri, vai in cv.split(x_train, x_train['build_num']):
        print("="*50)

        model = LGBMRegressor(boosting_type='gbdt',
                            objective='regression_l2', 
                            n_estimators=iterations,
                            max_depth=12,
                            learning_rate=0.01,
                            colsample_bytree=0.95,
                            subsample=1.0,
                            num_leaves=256,
                            reg_alpha=0.01,
                            reg_lambda=0.01,
                            n_jobs=-1,
                            random_state=42)     
        model.fit(x_train.iloc[tri], np.log1p(y_train[tri]), 
                eval_set=[(x_train.iloc[vai], np.log1p(y_train[vai]))], 
                eval_metric = lgbm_smape,
                early_stopping_rounds=patience,
                verbose = 3000,
                categorical_feature=cat_features)
        
        pred = model.predict(x_train.iloc[vai])
        pred = np.expm1(pred)
        score = SMAPE(y_train[vai],pred)
        print(f"{fold} FOLD SMAPE : ",score)
        smape_l2_score.append(score)
        models_l2.append(model)
        
        fold += 1

C:\Users\sangw\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\sangw\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\sangw\Anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\sangw\Anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridd

[3000]	valid_0's l2: 0.00255485	valid_0's SMAPE: 0.426943
[6000]	valid_0's l2: 0.00230427	valid_0's SMAPE: 0.398912
[9000]	valid_0's l2: 0.00220983	valid_0's SMAPE: 0.387095
[12000]	valid_0's l2: 0.0021662	valid_0's SMAPE: 0.380652
[15000]	valid_0's l2: 0.00214265	valid_0's SMAPE: 0.377449
[18000]	valid_0's l2: 0.00212623	valid_0's SMAPE: 0.37511
[21000]	valid_0's l2: 0.00211517	valid_0's SMAPE: 0.373478
[24000]	valid_0's l2: 0.00210781	valid_0's SMAPE: 0.372346
[27000]	valid_0's l2: 0.00210354	valid_0's SMAPE: 0.37165
[30000]	valid_0's l2: 0.00210069	valid_0's SMAPE: 0.371174
[33000]	valid_0's l2: 0.00209829	valid_0's SMAPE: 0.370786
[36000]	valid_0's l2: 0.00209622	valid_0's SMAPE: 0.370491
[39000]	valid_0's l2: 0.0020951	valid_0's SMAPE: 0.370316
[42000]	valid_0's l2: 0.00209428	valid_0's SMAPE: 0.370159
[45000]	valid_0's l2: 0.00209355	valid_0's SMAPE: 0.370038
[48000]	valid_0's l2: 0.00209307	valid_0's SMAPE: 0.369956
[51000]	valid_0's l2: 0.0020926	valid_0's SMAPE: 0.369877
[5400

[48000]	valid_0's l2: 0.00225203	valid_0's SMAPE: 0.374431
7 FOLD SMAPE :  2.6592661509561846
[3000]	valid_0's l2: 0.0025633	valid_0's SMAPE: 0.430069
[6000]	valid_0's l2: 0.00231512	valid_0's SMAPE: 0.402401
[9000]	valid_0's l2: 0.00222096	valid_0's SMAPE: 0.390691
[12000]	valid_0's l2: 0.00217208	valid_0's SMAPE: 0.384267
[15000]	valid_0's l2: 0.00214815	valid_0's SMAPE: 0.380763
[18000]	valid_0's l2: 0.00213184	valid_0's SMAPE: 0.378379
[21000]	valid_0's l2: 0.0021215	valid_0's SMAPE: 0.376825
[24000]	valid_0's l2: 0.00211488	valid_0's SMAPE: 0.37575
[27000]	valid_0's l2: 0.00211015	valid_0's SMAPE: 0.375002
[30000]	valid_0's l2: 0.00210656	valid_0's SMAPE: 0.374455
[33000]	valid_0's l2: 0.00210434	valid_0's SMAPE: 0.37408
[36000]	valid_0's l2: 0.00210271	valid_0's SMAPE: 0.373782
[39000]	valid_0's l2: 0.0021015	valid_0's SMAPE: 0.373586
[42000]	valid_0's l2: 0.00210069	valid_0's SMAPE: 0.373444
[45000]	valid_0's l2: 0.00209991	valid_0's SMAPE: 0.373315
[48000]	valid_0's l2: 0.00209

In [25]:
models_l1 = []

n_split_list = [10]
smape_l1_score = []

for split in n_split_list:
    cv = StratifiedKFold(n_splits=split, shuffle=True, random_state=42)
    
    fold = 1
    for tri, vai in cv.split(x_train, x_train['build_num']):
        print("="*50)
        
        model = LGBMRegressor(boosting_type='gbdt',
                            objective='regression_l1', 
                            n_estimators=iterations,
                            max_depth=12,
                            learning_rate=0.01,
                            colsample_bytree=0.95,
                            subsample=1.0,
                            num_leaves=256,
                            reg_alpha=0.01,
                            reg_lambda=0.01,
                            n_jobs=-1,
                            random_state=42)     
        model.fit(x_train.iloc[tri], np.log1p(y_train[tri]), 
                eval_set=[(x_train.iloc[vai], np.log1p(y_train[vai]))], 
                eval_metric = lgbm_smape,
                early_stopping_rounds=patience,
                verbose = 3000,
                categorical_feature=cat_features)

        pred = model.predict(x_train.iloc[vai])
        pred = np.expm1(pred)
        score = SMAPE(y_train[vai],pred)
        print(f"{fold} FOLD SMAPE : ",score)
        smape_l1_score.append(score)
        models_l1.append(model)
        
        fold += 1

C:\Users\sangw\Anaconda3\lib\site-packages\lightgbm\sklearn.py:726: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\sangw\Anaconda3\lib\site-packages\lightgbm\sklearn.py:736: UserWarning: 'verbose' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose' argument is deprecated and will be removed in a future release of LightGBM. "
C:\Users\sangw\Anaconda3\lib\site-packages\lightgbm\basic.py:2065: UserWarning: Using categorical_feature in Dataset.
  _log_warning('Using categorical_feature in Dataset.')
C:\Users\sangw\Anaconda3\lib\site-packages\lightgbm\basic.py:2068: UserWarning: categorical_feature in Dataset is overridd

[3000]	valid_0's l1: 0.0344541	valid_0's SMAPE: 0.483385
[6000]	valid_0's l1: 0.0329947	valid_0's SMAPE: 0.463189
[9000]	valid_0's l1: 0.0322274	valid_0's SMAPE: 0.45272
[12000]	valid_0's l1: 0.031784	valid_0's SMAPE: 0.446613
[15000]	valid_0's l1: 0.0315057	valid_0's SMAPE: 0.442775
[18000]	valid_0's l1: 0.0312433	valid_0's SMAPE: 0.439243
[21000]	valid_0's l1: 0.03104	valid_0's SMAPE: 0.436423
[24000]	valid_0's l1: 0.0308475	valid_0's SMAPE: 0.433755
[27000]	valid_0's l1: 0.0306317	valid_0's SMAPE: 0.430793
[30000]	valid_0's l1: 0.0304455	valid_0's SMAPE: 0.428227
[33000]	valid_0's l1: 0.0303197	valid_0's SMAPE: 0.42648
[36000]	valid_0's l1: 0.030218	valid_0's SMAPE: 0.424967
[39000]	valid_0's l1: 0.0300706	valid_0's SMAPE: 0.422847
[42000]	valid_0's l1: 0.0299777	valid_0's SMAPE: 0.421565
[45000]	valid_0's l1: 0.029848	valid_0's SMAPE: 0.419753
[48000]	valid_0's l1: 0.0297707	valid_0's SMAPE: 0.418666
[51000]	valid_0's l1: 0.0296906	valid_0's SMAPE: 0.417553
[54000]	valid_0's l1: 0.

[39000]	valid_0's l1: 0.0299645	valid_0's SMAPE: 0.421023
[42000]	valid_0's l1: 0.0298535	valid_0's SMAPE: 0.419489
[45000]	valid_0's l1: 0.0297558	valid_0's SMAPE: 0.418112
[48000]	valid_0's l1: 0.029678	valid_0's SMAPE: 0.416972
[51000]	valid_0's l1: 0.0295714	valid_0's SMAPE: 0.415356
[54000]	valid_0's l1: 0.0295103	valid_0's SMAPE: 0.414274
[57000]	valid_0's l1: 0.0294707	valid_0's SMAPE: 0.413704
[60000]	valid_0's l1: 0.0293831	valid_0's SMAPE: 0.412543
[63000]	valid_0's l1: 0.0293443	valid_0's SMAPE: 0.411892
[66000]	valid_0's l1: 0.0293051	valid_0's SMAPE: 0.411286
[69000]	valid_0's l1: 0.0292613	valid_0's SMAPE: 0.410684
[72000]	valid_0's l1: 0.0292241	valid_0's SMAPE: 0.410192
[75000]	valid_0's l1: 0.02919	valid_0's SMAPE: 0.409702
[78000]	valid_0's l1: 0.0291718	valid_0's SMAPE: 0.40946
6 FOLD SMAPE :  2.914982274724576
[3000]	valid_0's l1: 0.0343262	valid_0's SMAPE: 0.48357
[6000]	valid_0's l1: 0.0328812	valid_0's SMAPE: 0.463562
[9000]	valid_0's l1: 0.0321296	valid_0's SMAP

## 4. test inference

In [26]:
from tqdm import tqdm

preds_l2 = []
for i in tqdm(range(10)):
    pred = models_l2[i].predict(x_test)
    pred = np.expm1(pred)
    preds_l2.append(pred)
    
preds_l2 = np.mean(preds_l2 , axis = 0)

preds_l1 = []
for i in tqdm(range(10)):
    pred = models_l1[i].predict(x_test)
    pred = np.expm1(pred)
    preds_l1.append(pred)
    
preds_l1 = np.mean(preds_l1 , axis = 0)

100%|█████████████████████████████████████████████████████████████████████████████████| 10/10 [26:44<00:00, 160.49s/it]


In [27]:
preds = (preds_l2 + preds_l1) / 2

In [28]:
submission = pd.read_csv('./sample_submission.csv')
submission['answer'] = preds
submission.to_csv('./submission/LGBM_log_ensemble_test.csv', index = False)